# Introduccion

Para el ejercicio 2 del Trabajo Práctico se ha elegido utilizar la especificación OpenMP \[1\] para paralelizar el algoritmo de ordenamiento Quicksort \[2\] para ser aplicado en arreglos de gran tamaño.

El ordenamiento Quicksort es uno de aquellos algoritmos que Kernighan & Donovan han descripto como "embarasozamente paralelos" \[3\], esto es, algoritmos cuya implementación paralela es directa, y es muy sencillo disfrutar de los beneficios de la paralelización.

Basandonos en \[4\] hemos desarrollado una implementación de la paralelización de dicho algoritmo.

# Armado del ambiente

No es necesario ejecutar ningun comando previo a la ejecución de los programas.

# Desarrollo CPU (Python)

In [1]:
import numpy as np
import timeit

def partition(arr, first, last):
    i = first-1
    pivot = arr[last]

    for j in range(first, last):
      if arr[j] <= pivot:
        i += 1
        arr[i], arr[j] = arr[j], arr[i]
    
    arr[i+1], arr[last] = arr[last], arr[i+1]
    return i

def quicksort(arr, first, last):
  if first < last:
    i = partition(arr, first, last)
    quicksort(arr, first, i)
    quicksort(arr, i+1, last)

n = 10000000
a = []
def qs():
  quicksort(a, 0, n-1)

print(f"Aplicando quicksort secuencial en Python para un arreglo con {n} elementos")
for i in range(1, 16):
  a = np.random.rand(n)
  ex = timeit.Timer(qs).repeat(repeat=1, number=1)
  print(f'Iteracion {i}: {ex} segundos')

Aplicando quicksort secuencial en Python para un arreglo con 10000000 elementos
Iteracion 1: [150.13436613800002] segundos
Iteracion 2: [154.00084561] segundos
Iteracion 3: [153.77337166299998] segundos
Iteracion 4: [151.21775118] segundos
Iteracion 5: [156.28745900900003] segundos
Iteracion 6: [150.26272930699997] segundos
Iteracion 7: [152.2015946350001] segundos
Iteracion 8: [159.6803800099999] segundos
Iteracion 9: [152.69267990000003] segundos
Iteracion 10: [150.76703879899992] segundos
Iteracion 11: [150.91811322900003] segundos
Iteracion 12: [152.640539686] segundos
Iteracion 13: [150.274009686] segundos
Iteracion 14: [153.0860451200001] segundos
Iteracion 15: [156.6101902129999] segundos


# Desarrollo OpenMP

In [2]:
src = """
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <sys/time.h>

#define N 10000000

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[1];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0)\
    - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define OMP      0
// ----------------------------------------------------------------------------

int partition(int first, int last, int arr[])
{
    int j = last;
    int i = first-1;
    const int pivot = arr[last];

    while ( 1 ) {
      while ( arr[++i] < pivot )
        /* no op */;
      while ( j > 0 && arr[--j] > pivot )
        /* no op */;
      
      if ( i >= j )
        break;
      
      const int tmp = arr[i];
      arr[i] = arr[j];
      arr[j] = tmp;
    }
    
    const int tmp = arr[i];
    arr[i] = arr[last];
    arr[last] = tmp;

    return i;
}

void parallelQS(int first, int last, int arr[])
{
  if ( first < last ) {
    const int i = partition(first, last, arr);
    #pragma omp parallel sections
    {
      #pragma omp section
      parallelQS(first, i-1, arr);
      #pragma omp section
      parallelQS(i+1, last, arr);
    }
  }
}

int main(void)
{
  srand(time(NULL));

  int *arr = malloc(sizeof(int)*N);
  for ( int i = 0; i < N; ++i ) {
    arr[i] = rand();
  }
  
  TIEMPO_INI(OMP);
  parallelQS(0, N-1, arr);
  TIEMPO_FIN(OMP);
  printf("%lf miliseconds\\n", TIEMPO_GET(OMP));
}
"""
f = open("code.c", "w")
f.write(src)
f.close()
!gcc -o qs -fopenmp code.c

In [4]:
%env OMP_NUM_THREADS=1
print("Para un hilo")
for i in range(0, 15):
  print(f"Iteracion {i+1}: ", end="")
  !./qs

env: OMP_NUM_THREADS=1
Para un hilo
Iteracion 1: 6749.592781 miliseconds
Iteracion 2: 6747.409105 miliseconds
Iteracion 3: 6743.854046 miliseconds
Iteracion 4: 6813.382149 miliseconds
Iteracion 5: 6820.693970 miliseconds
Iteracion 6: 6669.458866 miliseconds
Iteracion 7: 6782.799006 miliseconds
Iteracion 8: 6984.551907 miliseconds
Iteracion 9: 6824.025869 miliseconds
Iteracion 10: 7048.274040 miliseconds
Iteracion 11: 7093.243122 miliseconds
Iteracion 12: 7036.939144 miliseconds
Iteracion 13: 6925.175190 miliseconds
Iteracion 14: 6873.319149 miliseconds
Iteracion 15: 6799.598932 miliseconds


In [5]:
%env OMP_NUM_THREADS=2
print("Para dos hilos")
for i in range(0, 15):
  print(f"Iteracion {i+1}: ", end="")
  !./qs

env: OMP_NUM_THREADS=2
Para dos hilos
Iteracion 1: 6646.591902 miliseconds
Iteracion 2: 6497.903109 miliseconds
Iteracion 3: 6716.470003 miliseconds
Iteracion 4: 6704.761982 miliseconds
Iteracion 5: 6484.121084 miliseconds
Iteracion 6: 6689.860106 miliseconds
Iteracion 7: 5785.536051 miliseconds
Iteracion 8: 6866.939068 miliseconds
Iteracion 9: 6748.593092 miliseconds
Iteracion 10: 6592.689991 miliseconds
Iteracion 11: 5694.109917 miliseconds
Iteracion 12: 6086.859941 miliseconds
Iteracion 13: 6957.990885 miliseconds
Iteracion 14: 6762.070179 miliseconds
Iteracion 15: 6073.780060 miliseconds


In [6]:
%env OMP_NUM_THREADS=4
print("Para cuatro hilos")
for i in range(0, 15):
  print(f"Iteracion {i+1}: ", end="")
  !./qs

env: OMP_NUM_THREADS=4
Para cuatro hilos
Iteracion 1: 6236.574888 miliseconds
Iteracion 2: 6789.102077 miliseconds
Iteracion 3: 6958.543062 miliseconds
Iteracion 4: 6510.991096 miliseconds
Iteracion 5: 6389.163017 miliseconds
Iteracion 6: 6371.634960 miliseconds
Iteracion 7: 5764.180899 miliseconds
Iteracion 8: 6251.686096 miliseconds
Iteracion 9: 6381.626129 miliseconds
Iteracion 10: 6732.842922 miliseconds
Iteracion 11: 5610.378027 miliseconds
Iteracion 12: 5842.894077 miliseconds
Iteracion 13: 6516.905069 miliseconds
Iteracion 14: 6312.066078 miliseconds
Iteracion 15: 6378.349066 miliseconds


In [7]:
%env OMP_NUM_THREADS=8
print("Para ocho hilos")
for i in range(0, 15):
  print(f"Iteracion {i+1}: ", end="")
  !./qs

env: OMP_NUM_THREADS=8
Para ocho hilos
Iteracion 1: 6390.315056 miliseconds
Iteracion 2: 6450.614929 miliseconds
Iteracion 3: 5861.887932 miliseconds
Iteracion 4: 6882.541895 miliseconds
Iteracion 5: 6970.826149 miliseconds
Iteracion 6: 6670.109987 miliseconds
Iteracion 7: 5647.617102 miliseconds
Iteracion 8: 5709.305048 miliseconds
Iteracion 9: 6144.018173 miliseconds
Iteracion 10: 6651.249886 miliseconds
Iteracion 11: 6674.711943 miliseconds
Iteracion 12: 6533.761024 miliseconds
Iteracion 13: 5998.939991 miliseconds
Iteracion 14: 6662.028074 miliseconds
Iteracion 15: 5650.175095 miliseconds


# Métricas

Los resultados obtenidos son los siguientes.

## Resultados para ejecucion Python

| Iteracion | Tiempo (segundos)  |
| -:        | -:                 |
| 1         | 147.80152964399895 |
| 2         | 151.38177975500003 |
| 3         | 151.88150012799997 |
| 4         | 145.55940450799972 |
| 5         | 148.9144810580001  |
| 6         | 156.96874078899964 |
| 7         | 147.57664608400046 |
| 8         | 151.58829699600028 |
| 9         | 150.63702954399923 |
| 10        | 153.25324131600064 |
| 11        | 149.2906379879987  |
| 12        | 156.94186678699953 |
| 13        | 153.73340939099944 |
| 14        | 151.42952871599846 |
| 15        | 150.63702954399923 |
| **Promedio** | 151.17300814987 |

## Resultados para ejecucion con 1 hilo

| Iteracion | Tiempo (milisegundos)  |
| -:        | -:                     |
| 1 | 6876.526117 |
| 2 | 6964.706898 |
| 3 | 7038.475990 |
| 4 | 7015.191078 |
| 5 | 7027.406931 |
| 6 | 7129.604101 |
| 7 | 6984.683037 |
| 8 | 6872.027159 |
| 9 | 6995.491028 |
| 10 | 6942.981958 |
| 11 | 7010.185003 |
| 12 | 6964.138985 |
| 13 | 7009.713888 |
| 14 | 6922.940969 |
| 15 | 7070.497990 |
| **Promedio** | 6988.3047421333 |

## Resultados para ejecucion con 2 hilos

| Iteracion | Tiempo (milisegundos)  |
| -:        | -:                     |
| 1 | 5613.468885 |
| 2 | 6913.333178 |
| 3 | 5804.399967 |
| 4 | 5694.590092 |
| 5 | 6820.485115 |
| 6 | 6116.336107 |
| 7 | 6499.118805 |
| 8 | 6185.397148 |
| 9 | 5559.990168 |
| 10 | 5742.691040 |
| 11 | 6167.668104 |
| 12 | 6346.008062 |
| 13 | 6520.165920 |
| 14 | 6213.366032 |
| 15 | 6321.040869 |
| **Promedio** | 6167.8706328 |

## Resultados para ejecucion con 4 hilos

| Iteracion | Tiempo (milisegundos)  |
| -:        | -:                     |
| 1 | 5744.879961 |
| 2 | 5791.626930 |
| 3 | 6516.236067 |
| 4 | 5938.782930 |
| 5 | 6702.416897 |
| 6 | 6277.110815 |
| 7 | 5920.520067 |
| 8 | 5599.580050 |
| 9 | 5853.758812 |
| 10 | 6506.923914 |
| 11 | 6816.364050 |
| 12 | 6892.578840 |
| 13 | 5673.367023 |
| 14 | 5667.171001 |
| 15 | 6481.529951 |
| **Promedio** | 6158.8564872 |

## Resultados para ejecucion con 8 hilos

| Iteracion | Tiempo (milisegundos)  |
| -:        | -:                     |
| 1 | 5869.745970 |
| 2 | 6604.358912 |
| 3 | 5975.147009 |
| 4 | 5918.689966 |
| 5 | 6294.317961 |
| 6 | 6784.985065 |
| 7 | 6053.102970 |
| 8 | 5751.697779 |
| 9 | 6064.453125 |
| 10 | 6858.651876 |
| 11 | 5562.403917 |
| 12 | 5770.869970 |
| 13 | 5657.296896 |
| 14 | 6387.918949 |
| 15 | 6467.962980 |
|**Promedio**|6134.7735563333|

# Conclusiones

De los resultados podemos extraer una serie de conclusiones:

1. La diferencia entre la ejecución del mismo algoritmo utilizando un lenguaje interpretado (Python) y un lenguaje compilado (C), con la misma cantidad de hilos en ejecución, implica una mejora de velocidad de hasta 21 veces.
2. Utilizar paralelización implica un speed up de $1.13301$.
3. Aumentar la cantidad de hilos (de 2 hacia 4 y 8) no necesariamente aumenta una gran cantidad la velocidad del proceso. Esto se debe a que el entorno de Google Colab cuenta con tan sólo 2 procesadores, y por lo tanto, ejecutar más de 2 hilos no ofrece una nueva dimensión de paralelismo, sino que tan sólo aumenta la concurrencia. El speed up que se ve cuando se utilizan 4 hilos es de $1.134675$ (apenas $0.0016$ más que con 2 hilos), y cuando se utilizan 8 hilos es de $1.13913$ (apenas $0.006$ que con 2 hilos).

# Bibliografía

1. OpenMP, “OPENMP API Specification: Version 5.0,” OpenMP, Nov-2018. \[Online\]. Available: https://www.openmp.org/spec-html/5.0/openmp.html. \[Accessed: 22-Nov-2021\].
2. C. A. R. Hoare, Quicksort, The Computer Journal, Volume 5, Issue 1, 1962, pp. 10-16 https://doi.org/10.1093/comjnl/5.1.10
3. A. A. A. Donovan and B. W. Kernighan, “8.5 Looping In Parallel,” in The go programming language, New York, N.Y: Addison-Wesley, 2020, pp. 235–235.
4. Sinan Sameer Mahmood Al-Dabbagh et al, Int. Journal of Computer Science & Mobile Computing, Vol.5 Issue.6, June- 2016, pp. 372-382, https://www.radford.edu/~cshing/310/Lecture/PDC/parallelQuicksort.pdf
